In [1]:
!pip install PyPDF2 sentence-transformers faiss-cpu openai numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 61.6 MB/s eta 0:00:00:00:0100:01


In [2]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

def chunk_text(text, chunk_size=500):
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
    return chunks

In [3]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

def generate_embeddings(chunks):
    return model.encode(chunks)

def build_faiss_index(embeddings):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

def save_faiss_index(index, file_path):
    faiss.write_index(index, file_path)

def load_faiss_index(file_path):
    return faiss.read_index(file_path)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
from sentence_transformers import SentenceTransformer

def retrieve_chunks(query, index, chunks, k=3):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, k)
    return [chunks[i] for i in indices[0]]

In [5]:
from transformers import pipeline

# Khởi tạo pipeline tóm tắt với Hugging Face
summarizer = pipeline("summarization", model="openai-community/gpt2")

def summarize_with_llm(text):
    # Tóm tắt văn bản
    summary = summarizer(text, max_length=512, min_length=30, do_sample=False, temperature=0.7)
    return summary[0]['summary_text']

# Test với đoạn văn bản
text = "Machine learning is a method of data analysis that automates analytical model building..."
summary = summarize_with_llm(text)
print(summary)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
The model 'GPT2LMHeadModel' is not supported for summarization. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].
Yo

Machine learning is a method of data analysis that automates analytical model building...

The Future of Machine Learning

Machine learning is a method of data analysis that automates analytical model building. It is a way to build a model that is more accurate than the human eye can see. It is a way to build a model that is more accurate than the human eye can see. It is a way to build a model that is more accurate than the human eye can see. It is a way to build a model that is more accurate than the human eye can see. It is a way to build a model that is more accurate than the human eye can see. It is a way to build a model that is more accurate than the human eye can see. It is a way to build a model that is more accurate than the human eye can see. It is a way to build a model that is more accurate than the human eye can see. It is a way to build a model that is more accurate than the human eye can see. It is a way to build a model that is more accurate than the human eye can see.

In [6]:
def chat_with_pdf(query, index, chunks):
    relevant_chunks = retrieve_chunks(query, index, chunks)
    context = " ".join(relevant_chunks)
    summary = summarize_with_llm(context)
    return summary

In [7]:
import os

# Paths
pdf_folder = "/kaggle/input/data-pdf"
index_path = "/kaggle/working/pdf_index.faiss"

# Step 1: Extract and chunk text
pdf_path = os.path.join(pdf_folder, "example.pdf")
text = extract_text_from_pdf(pdf_path)
chunks = chunk_text(text)

# Step 2: Generate embeddings and build FAISS index
embeddings = generate_embeddings(chunks)
index = build_faiss_index(embeddings)
save_faiss_index(index, index_path)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
query = "Summary of this pdf file"
summary = chat_with_pdf(query, index, chunks)
print("Summary:", summary)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Your max_length is set to 512, but your input_length is only 426. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=213)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Summary: rer would like to prove that it has manufactured the
product. Anyone can freely identify the manufacturer of the
product by viewing the ﬁrst owner recorded in PMC .
C. Distribution management
Fig. 2 illustrates the ﬂow of distribution management.
Distribution management consists of the following eight st eps:
Steps 1 to 4 are the shipping process and Steps 5 to 8 are the
receiving process.
1) The owner shares a secret token with the recipient by a
secure method.
2) The owner encrypts the recipie hereum.org, [Online; accessed
17-March-2020].
[11] “ZoKrates,” https://github.com/Zokrates/ZoKrates , [Online; accessed 17-
March-2020].
[12] OECD, “Enhancing product recall effectiveness global ly,”OECD Sci-
ence, Technology and Industry Policy Papers , no. 58, Nov. 2018. . Pinto, “An introduction to the use of zk-SNARKs in bl ockchains,”
inProceedings of Mathematical Research for Blockchain Econo my,
2020, pp. 233–249.
[8] B. WhiteHat, J. Baylina, and M. Bell´ es, “Baby jubjub ell ip